In [14]:
# Packages

%matplotlib inline
import numpy as np
#import dicom
#import pydicom as d

import os
import time 

import matplotlib as mt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras.utils.vis_utils import plot_model
from PIL import Image


from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

import tensorflow as tf

print(os.listdir('2000_images'))
tf.__version__

['test', 'train']


'2.8.0'

In [28]:
# Classification Model
CLASSIFY_INPUT_WIDTH, CLASSIFY_INPUT_HEIGHT, CLASSIFY_INPUT_DEPTH, CLASSIFY_INPUT_CHANNEL = 64,64,64,3

inputs = Input((CLASSIFY_INPUT_WIDTH, CLASSIFY_INPUT_HEIGHT, CLASSIFY_INPUT_DEPTH, CLASSIFY_INPUT_CHANNEL))


x = Conv3D(32, (3, 3, 3), padding='same', activation='relu')(inputs)
x = MaxPooling3D(pool_size=(2, 2, 2))(x)


x=Flatten()(x)
x = Dense(1, activation='sigmoid')(x)


y = Conv3D(64, (3, 3, 3), padding='same', activation='relu')(inputs)
y = MaxPooling3D(pool_size=(2, 2, 2))(y)


y=Flatten()(x)
y = Dense(1, activation='sigmoid')(y)

model = Model(inputs=inputs, outputs=(x))

model1 = Model(inputs=inputs, outputs=(y))

METRICS = [tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision(name="precision"), tf.keras.metrics.Recall(name="recall")]

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

model.summary()


model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

model1.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 64, 64, 64, 3)]   0         
                                                                 
 conv3d_4 (Conv3D)           (None, 64, 64, 64, 32)    2624      
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 32, 32, 32, 32)   0         
 3D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 1048576)           0         
                                                                 
 dense_4 (Dense)             (None, 1)                 1048577   
                                                                 
Total params: 1,051,201
Trainable params: 1,051,201
Non-trainable params: 0
_________________________________________________

In [29]:
# Train and Test the model
start= time.time()


train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory("2000_images/train",
                                                 target_size = (64,64,64),
                                                 batch_size = 8,
                                                 class_mode =  'binary')

test_set = test_datagen.flow_from_directory("2000_images/test",
                                            target_size = (64,64,64),
                                            batch_size = 8,
                                            class_mode =  'binary')


Found 1400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [31]:
res=model.fit(training_set, batch_size=3, steps_per_epoch = 10, epochs = 5, validation_data=test_set)

print("Done in %.2f s." % (time.time() - start))


Epoch 1/5
10/10 [==============================] - 122s 13s/step - loss: 15.3869 - binary_accuracy: 0.9074 - precision: 0.9074 - recall: 1.0000 - val_loss: 7.7650 - val_binary_accuracy: 0.9000 - val_precision: 0.9000 - val_recall: 1.0000
Epoch 2/5
10/10 [==============================] - 114s 12s/step - loss: 4.1526 - binary_accuracy: 0.6250 - precision: 0.8214 - recall: 0.6970 - val_loss: 1.7163 - val_binary_accuracy: 0.9000 - val_precision: 0.9000 - val_recall: 1.0000
Epoch 3/5
10/10 [==============================] - 116s 12s/step - loss: 1.1204 - binary_accuracy: 0.9125 - precision: 0.9125 - recall: 1.0000 - val_loss: 0.5013 - val_binary_accuracy: 0.9000 - val_precision: 0.9000 - val_recall: 1.0000
Epoch 4/5
10/10 [==============================] - 119s 12s/step - loss: 0.3391 - binary_accuracy: 0.9500 - precision: 0.9500 - recall: 1.0000 - val_loss: 0.3818 - val_binary_accuracy: 0.9000 - val_precision: 0.9000 - val_recall: 1.0000
Epoch 5/5
10/10 [==============================] - 

In [30]:
res1=model1.fit(training_set, batch_size=3, steps_per_epoch = 10, epochs = 5, validation_data=test_set)

print("Done in %.2f s." % (time.time() - start))

Epoch 1/5
10/10 [==============================] - 142s 15s/step - loss: 0.5962 - binary_accuracy: 0.9125 - precision: 0.9125 - recall: 1.0000 - val_loss: 0.5895 - val_binary_accuracy: 0.9000 - val_precision: 0.9000 - val_recall: 1.0000
Epoch 2/5
10/10 [==============================] - 142s 15s/step - loss: 0.5978 - binary_accuracy: 0.8625 - precision: 0.8625 - recall: 1.0000 - val_loss: 0.5836 - val_binary_accuracy: 0.9000 - val_precision: 0.9000 - val_recall: 1.0000
Epoch 3/5
10/10 [==============================] - 143s 15s/step - loss: 0.5807 - binary_accuracy: 0.9000 - precision: 0.9000 - recall: 1.0000 - val_loss: 0.5778 - val_binary_accuracy: 0.9000 - val_precision: 0.9000 - val_recall: 1.0000
Epoch 4/5
10/10 [==============================] - 162s 17s/step - loss: 0.5625 - binary_accuracy: 0.9375 - precision: 0.9375 - recall: 1.0000 - val_loss: 0.5715 - val_binary_accuracy: 0.9000 - val_precision: 0.9000 - val_recall: 1.0000
Epoch 5/5
10/10 [==============================] - 1

In [44]:
imdim = (64,64,64,3)
print('-'*60)                                              

enterimg = input("Enter the path of image ")   
test_image = image.load_img(enterimg)
print('-'*60)                                              

test_image = image.img_to_array(test_image).reshape((imdim[0],imdim[1],imdim[2],imdim[3]))
test_image = np.expand_dims(test_image, axis=0)

result = model.predict(test_image)
training_set.class_indices

result1 = model1.predict(test_image)
training_set.class_indices

                                             
                                             


------------------------------------------------------------
------------------------------------------------------------


{'0': 0, '1': 1}

In [ ]:
if result[0] == 0:
    prediction = 'Found'
    print('Lung Cancer Detected !')
else:
    print('Lung Cancer is not detected !')
    
print('-'*60)                                              



if result1[0] == 0:
    prediction = 'Found'
    print('Lung Cancer Detected !')
else:
    print('Lung Cancer is not detected !')
    
print('-'*60) 

In [45]:
# Analysis

print('\nModel 1 ----------------------')

train_pre = res.history['precision'][-1]
train_re = res.history['recall'][-1]
test_pre = res.history['precision'][-1]
test_re = res.history['recall'][-1]

print('\nTRAIN METRIC ----------------------')
print('Train Accuracy: {}'.format(np.round((res.history['binary_accuracy'][-1])*100, 2)))
print('Train Precision: {}'.format(np.round((train_pre)*100, 2)))
print('Train Recall: {}'.format(np.round((train_re)*100, 2)))
print('Train F1-score: {}'.format(np.round((2*(train_re * train_pre)/(train_re + train_pre))*100, 2)))

print('\nTEST METRIC ----------------------')
print('Test Accuracy: {}'.format(np.round((res.history['val_binary_accuracy'][-1])*100, 2)))
print('Test Precision: {}'.format(np.round((test_pre)*100, 2)))
print('Test Recall: {}'.format(np.round((test_re)*100, 2)))
print('Test F1-score: {}'.format(np.round((2*(test_re * test_pre)/(test_re + test_pre))*100, 2)))


print('\nModel 2 ----------------------')


train_pre = res1.history['precision'][-1]
train_re = res1.history['recall'][-1]
test_pre = res1.history['precision'][-1]
test_re = res1.history['recall'][-1]

print('\nTRAIN METRIC ----------------------')
print('Train Accuracy: {}'.format(np.round((res1.history['binary_accuracy'][-1])*100, 2)))
print('Train Precision: {}'.format(np.round((train_pre)*100, 2)))
print('Train Recall: {}'.format(np.round((train_re)*100, 2)))
print('Train F1-score: {}'.format(np.round((2*(train_re * train_pre)/(train_re + train_pre))*100, 2)))

print('\nTEST METRIC ----------------------')
print('Test Accuracy: {}'.format(np.round((res1.history['val_binary_accuracy'][-1])*100, 2)))
print('Test Precision: {}'.format(np.round((test_pre)*100, 2)))
print('Test Recall: {}'.format(np.round((test_re)*100, 2)))
print('Test F1-score: {}'.format(np.round((2*(test_re * test_pre)/(test_re + test_pre))*100, 2)))




Model 1 ----------------------

TRAIN METRIC ----------------------
Train Accuracy: 90.0
Train Precision: 90.0
Train Recall: 100.0
Train F1-score: 94.74

TEST METRIC ----------------------
Test Accuracy: 90.0
Test Precision: 90.0
Test Recall: 100.0
Test F1-score: 94.74

Model 2 ----------------------

TRAIN METRIC ----------------------
Train Accuracy: 88.75
Train Precision: 88.75
Train Recall: 100.0
Train F1-score: 94.04

TEST METRIC ----------------------
Test Accuracy: 90.0
Test Precision: 88.75
Test Recall: 100.0
Test F1-score: 94.04


In [46]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=100,random_state=456)

In [61]:

training_set1 = train_datagen.flow_from_directory("2000_images/train",
                                                 target_size = (8,8,8),
                                                 batch_size = 8,
                                                 class_mode =  'binary')

test_set1 = test_datagen.flow_from_directory("2000_images/test",
                                            target_size = (8,8,8),
                                            batch_size = 8,
                                            class_mode =  'binary')

Found 1400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [62]:
ada.fit(training_set1, test_set1)
ada.score(training_set1, test_set1)

